In [3]:
import matplotlib.pyplot as plt
plt.rc('font', family="NanumBarunGothic")

In [4]:
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from transformers import MarianMTModel, MarianTokenizer # translation machine
import pandas as pd
from tqdm import tqdm
import math, random
from einops import rearrange

DEVICE = "GPU" if torch.cuda.is_available() else "CPU"
print(DEVICE)

c:\Users\hoya9\anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU


In [5]:
# for reproducibility
random_seed = 0
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # Multi-GPU case
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(random_seed)

In [6]:
# Load the tokenizer & model
tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en')
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ko-en') # MT: Machine Translation

In [7]:
eos_idx = tokenizer.eos_token_id
pad_idx = tokenizer.pad_token_id
print('eos_idx = ', eos_idx)
print('pad_idx = ', pad_idx)

eos_idx =  0
pad_idx =  65000


### 하이퍼파라미터 조정

In [8]:
BATCH_SIZE = 64 # 실제 논문에선 2.5만 token이 한 batch에 담기게 했다고 함
LAMBDA = 0 # l2-Regularization를 위한 hyperparams (저장된 모델)
EPOCH = 15 # 저장된 모델
max_len = 100 # 길이 제한 (GPU 부담도 많이 감소)
"""
decoder에서 모델이 <eos>가 출력되는 것까지는 Loss를 구하고,
그뒤 <pad>에 대한 Loss는 무시하기 위한 역활
즉, label이 <pad> 일 때는 무시
"""
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

scheduler_name = 'Noam' # Transformer 논문 제 2저자가 제안한 learning rate scheduler
scheduler_name = 'Cos'
#### Noam ####
# warmup_steps = 4000 # 실제 논문에서 제시한 값 (총 10만 step의 4%)
warmup_steps = 1000 # 데이터 수 * EPOCH / BS = 총 step 수 인것 고려 (저장된 모델)
LR_scale = .5 # Noam scheduler에 peak LR 값 조절을 위해 곱해질 값 (저장된 모델)
#### Cos ####
LR_init = 5e-4
T0 = 1500 # 첫 주기
T_mult = 2 # 배 만큼 주기가 길어짐 (1보다 큰 정수여야 함)
#############

"""
나만의 모델을 만들고 싶으면,
new_model_train = True
prev_model_use = False
"""
new_model_train = False
prev_model_use = True

!mkdir data

if prev_model_use:
    !gdown https://drive.google.com/uc?id=1bjbeWgqlVKJ9gzDcL1hfD9cIItQy9_N2 -O data/Transformer_small.pt
    !gdown https://drive.google.com/uc?id=1M0yYP2umxlwaAbk_iq5G_Z5y3qLu9Wet -O data/Transformer_small_history.pt

    save_model_path = 'data/Transformer_small.pt'
    save_history_path = 'data/Transformer_small_history.pt'
else:
    save_model=path = 'data/Transformer_small2.pt'
    save_history_path = 'data/Transformer_small2_history.pt'

A subdirectory or file data already exists.
Downloading...
From (original): https://drive.google.com/uc?id=1bjbeWgqlVKJ9gzDcL1hfD9cIItQy9_N2
From (redirected): https://drive.google.com/uc?id=1bjbeWgqlVKJ9gzDcL1hfD9cIItQy9_N2&confirm=t&uuid=32475125-3a81-4844-8f38-6bd97fb6adb5
To: c:\Users\hoya9\OneDrive\Desktop\torch-gpu\Workspace\Transformer-pytorch\data\Transformer_small.pt

  0%|          | 0.00/448M [00:00<?, ?B/s]
  0%|          | 524k/448M [00:00<03:31, 2.12MB/s]
  0%|          | 1.57M/448M [00:00<01:32, 4.83MB/s]
  1%|          | 2.62M/448M [00:00<01:16, 5.83MB/s]
  1%|          | 4.19M/448M [00:00<00:55, 8.03MB/s]
  1%|▏         | 6.29M/448M [00:00<00:41, 10.7MB/s]
  2%|▏         | 7.86M/448M [00:00<00:38, 11.5MB/s]
  2%|▏         | 9.96M/448M [00:01<00:35, 12.5MB/s]
  3%|▎         | 11.5M/448M [00:01<00:34, 12.7MB/s]
  3%|▎         | 13.6M/448M [00:01<00:37, 11.6MB/s]
  3%|▎         | 15.2M/448M [00:01<00:35, 12.3MB/s]
  4%|▍         | 17.3M/448M [00:01<00:29, 14.4MB/s]
  4%|▍

c:\Users\hoya9\anaconda3\envs\torch-gpu\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hoya9\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-ko-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
"""
논문에 나오는 BASE MODEL
train loss를 많이 줄이려면 많은 epoch이 요구됨.
test 성능도 높이려면 더 많은 데이터가 필요함
"""
n_layers = 6
d_model = 512
d_ff = 2048
n_heads = 8
drop_p = .1

# 좀 사이즈 줄인 모델 (훈련된 input_embedding, fc_out 사용하면 사용 불가)
n_layers = 3
d_model = 26
d_ff = 512
n_heads = 8
drop_p = .1

### 토크나이저 & 학습된 모델 써보기

In [11]:
# tokenizer 써보기 (_로 띄어쓰기를 나타낸다! 즉, _가 없으면 이어진 한 단어 : subword tokenizing)
# tokenizer에 대한 참고 자료: https://ratsgo.github.io/nlpbook/docs/preprocess/bpe/
print(tokenizer.tokenize("Hi, I'm Ian. ... a  a?"))
print(tokenizer.tokenize('a/b 1+2+3 2:1 a>b'))
print(tokenizer.tokenize('pretrained restart'))
print(tokenizer.tokenize('chatGPT'))
print(tokenizer.tokenize('The example is very good in our lecture'))
print(tokenizer.tokenize('한글은 어떻게 할까?'))
print(tokenizer.tokenize('확실히 띄어쓰기 기준으로 토크나이징을 하는 것 같진 않다.'))
print(tokenizer.tokenize('여러분 차례!'))

['▁H', 'i', ',', '▁I', "'", 'm', '▁I', 'an', '.', '▁', '...', '▁a', '▁a', '?']
['▁a', '/', 'b', '▁1', '+2', '+3', '▁2:', '1', '▁a', '>', 'b']
['▁p', 're', 'tra', 'in', 'ed', '▁re', 'st', 'art']
['▁ch', 'at', 'G', 'P', 'T']
['▁The', '▁', 'ex', 'am', 'ple', '▁is', '▁', 'very', '▁good', '▁in', '▁', 'our', '▁', 'le', 'c', 'ture']
['▁한', '글', '은', '▁어떻게', '▁할까', '?']
['▁확실히', '▁띄', '어', '쓰기', '▁기준으로', '▁토', '크', '나이', '징', '을', '▁하는', '▁것', '▁같', '진', '▁않다', '.']
['▁여러분', '▁차례', '!']


In [16]:
# print(tokenizer.get_vocab()) # tokenizer에 들어있는 모든 토큰과 토큰의 해당하는 번호를 dict으로 가짐
vocab_size = tokenizer.vocab_size
print((vocab_size))

# add_special_token은 <eos> 자동 붙여주는 것을 방지
print(tokenizer.encode('지능', add_special_tokens=False))   # string to index
print(tokenizer.encode('<pad>', add_special_tokens=False))  # <pad>는 65000
print(tokenizer.encode('</s>', add_special_tokens=False))   # <sos> or <eos>는 0
print(tokenizer.encode('He', add_special_tokens=False))
print(tokenizer.encode('he', add_special_tokens=False))     # 대소문자 다른 단어로 인식
print(tokenizer.tokenize('문장을 넣으면 tokenize해서 숫자로 바꾼다'))
print(tokenizer.encode('문장을 넣으면 tokenize해서 숫자로 바꾼다', add_special_tokens=False))
print(tokenizer.decode([204]))
print(tokenizer.decode([204]))
print(tokenizer.decode([204]))
print(tokenizer.decode(list(range(15)) + [65000, 65001, 65002, 65003]))

65001
[34359]
[65000]
[0]
[125]
[59]
['▁문장', '을', '▁넣으면', '▁to', 'k', 'en', 'ize', '해서', '▁숫자', '로', '▁바꾼', '다']
[13774, 51, 40068, 5, 1479, 1252, 5016, 969, 6635, 131, 30737, 161]
사람
사람
사람
</s> <unk> ., the to of? and s a' in 들<pad> <unk> <unk> <unk>
